# Basic scraping for Guardian discussions u

In [18]:
! ./.g


[master 75ebeea] web
 1 file changed, 1914 deletions(-)
 delete mode 100644 tt.csv
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 4 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 213 bytes | 213.00 KiB/s, done.
Total 2 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/macbuse/GUARDIAN.git
   4d8b160..75ebeea  master -> master


In [4]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [9]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','w') as fp:
    fp.write('\n'.join(lks))

In [1]:
import re 
import requests
import pickle
import json

In [20]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/21/joe-biden-democrat-convention-nice-guy-trump-fight'

In [21]:
key = my_keys.keys['guardian']

NameError: name 'my_keys' is not defined

In [22]:
#scrape the page and get the short url
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)

#the short url allows us to access the discussion
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn ,'wb') as fp:
    pickle.dump(json_data,fp)
    


getting comments page no. 1
there are 3 pages
getting comments page no. 2
getting comments page no. 3
archiving to ej7f5.pkl


In [2]:
json_data = pickle.load(open('dpb6e.pkl','rb') ) #jenkins

archiving to dqhaa.pkl #jenkins

archiving to evdjz.pkl #big cummings

e9j7e.pkl # bragg

eghyd.pkl # migrants

SyntaxError: invalid syntax (<ipython-input-2-7a525c39f9dc>, line 3)

In [3]:
json_data = pickle.load(open('etenk.pkl','rb') ) #jenkins


## remember each page of comments has been stored separately

In [4]:
dd = json.loads(json_data[0])
len(json_data)

8

In [8]:
xx = json.loads(json_data[0])
xx.keys()

dict_keys(['postedCommentHtml', 'commentsHtml', 'commentCount', 'refreshStatus', 'lastPage', 'paginationHtml'])

In [9]:
print( xx['commentsHtml'][:100])



<div class="d-discussion d-discussion--recommendations-open u-cf"
     data-read-only="false" >

 


In [23]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')

auths = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh) , pid.findall(hh)) ))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [24]:
auths = dict(auths)

In [25]:
class Comment():
    pass

from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    auth = [x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    
    return [ '##{}\n{}\n\n{}'.format(a,b,c.strip())  
                                 for a,b,c in zip(auth, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]



from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

with open('b7jtb.pkl','rb') as fp:
    json_data = pickle.load(fp)

In [26]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)

In [27]:
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##CAROLEFURKINBASKIN', 9),
 ('##NOVEDSTHEBEST', 9),
 ('##SAGARMATHA1953', 7),
 ('##JAGSFAN80', 7),
 ('##ANGRYINSOCAL', 6),
 ('##PARTOFTHEPROBLEM', 6),
 ('##BREXITSBANE', 6),
 ('##JONBRADSHAW2020', 6)]

In [28]:
all_comments.sort(key=lambda x : -len(x) )

In [29]:
print( '\n\n'.join(all_comments[:20]) )

##PARTOFTHEPROBLEM
https://profile.theguardian.com/user/id/16171854

I missed the balloons, the streamers, the strutting around the stage. I missed the lines carefully calibrated for applause and the veering off-script. I missed the theatricality.

Why?! The above might be entertaining to you & others, but it's shouldn't have a place in politics as it places showmanship above everything else.

The convention felt like a charity marathon without an audience, punctuated with unfunny comedy and sob stories.

I've seen some suggest that the DNC didn't want to waste money when they have no faith in Biden/Kamala to win against Trump...I'm not sure I agree, but seeing that Billy Porter music video at the end of the first night they certainly didn't put a lot of effort into it.

The US is standing on the faultline of a pandemic, a deepening recession and a racial divide...

The racial divide seems to be driven in part by the media and activists who rush to sensationalise any incident they can 

In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [41]:
target = 'decolon'
target = 'science'
target = 'Obama^\s*'


xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )


#print( '\n\n'.join(xx).replace(target, ) )
pxx = re.compile(target, re.I)
print( pxx.sub('**' + target.upper() + '**', 
              '\n\n'.join(xx)) )
               
#print( '\n\n'.join(xx) )


/home/gregmcshane/.env/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: bad escape \S
  from ipykernel import kernelapp as app


36